# Analiza korisnika i zadataka na platformi RingZer0

In [1]:
import pandas as pd
import requests

In [2]:
# Funkcija za zvati api sa kompletnim error handlingom
import time

def call_api(url):
    for attempt in range(3): 
        try:
            response = requests.get(url, timeout=5)
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:  # Rate Limit
                time.sleep(2 ** attempt)
            else:
                time.sleep(1)
        except Exception as e:
            print(f"API call failed: {e}")
            time.sleep(2 ** attempt)  # Exponential backoff
    return None

## Prikupljanje podataka o kategorijama

Prvo želimo prikupiti sve kategorije u RingZero i prebaciti ih u standardne jeopardy kategorije.

In [ ]:
# Prikupi info o kategorijama 
url = "https://ringzer0ctf.com/api/categories"
response = requests.get(url)
data = response.json()

# Prikupi sve category id-jeve
categories = data['data']['categories']
category_ids = [cat['category']['id'] for cat in categories]
category_titles = [cat['category']['title'] for cat in categories]
print(f"Found {len(category_ids)} categories: {category_ids}")  
print(f"Category titles: {category_titles}")

Found 21 categories: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21']
Category titles: ['SQL Injection', 'Reverse Engineering', 'Steganography', 'Forensics', 'Coding Challenges', 'Cryptography', 'JavaScript', 'Web', 'Pwnage Linux', 'SysAdmin Linux', 'Jail Escaping', 'Shellcoding', 'Malware Analysis', 'The NC8 Reverse Engineering Track', 'Exotic Data Storage', 'Software Defined Radio', 'NorthSec 2021', 'NorthSec 2023', 'NorthSec 2022', 'NorthSec 2024', 'NorthSec 2025']


In [3]:
category_ids = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21']
category_titles = ['SQL Injection', 'Reverse Engineering', 'Steganography', 'Forensics', 'Coding Challenges', 'Cryptography', 'JavaScript', 'Web', 'Pwnage Linux', 'SysAdmin Linux', 'Jail Escaping', 'Shellcoding', 'Malware Analysis', 'The NC8 Reverse Engineering Track', 'Exotic Data Storage', 'Software Defined Radio', 'NorthSec 2021', 'NorthSec 2023', 'NorthSec 2022', 'NorthSec 2024', 'NorthSec 2025']

categories = {"Web": ["SQL Injection", "JavaScript", "Web"],
              "Rev": ["Reverse Engineering", "The NC8 Reverse Engineering Track"], 
              "Pwn": ["Pwnage Linux", "Jail Escaping", "Shellcoding", "Malware Analysis"], 
              "Crypt": ["Cryptography"], 
              "Forensics": ["Forensics", "Exotic Data Storage", "Steganography"], 
              "Misc": ["Software Defined Radio", "SysAdmin Linux", "Coding Challenges"], 
              "Special": ["NorthSec 2021", "NorthSec 2022", "NorthSec 2023", "NorthSec 2024", "NorthSec 2025"]}

Struktura tablica za podatke

In [ ]:
# Tables content
# users_df = pd.DataFrame(columns=["id", "username", "isRCEH", "country", "joined_date", "last_date", "points"	"rank"]) 
# challenges_df = pd.DataFrame(columns=["category_id", "category_title", "challenge_id", "challenge_title", "points"])
# solves_df = pd.DataFrame(columns=["user_id", "challenge_id", "solve_date"])

## Prikupljanje postojećih challenga na stranici

In [ ]:
# 419 chals 11.9.2025.
challenge_list = []
for cat_id in category_ids:
    url = f"https://ringzer0ctf.com/api/category/challenges/{cat_id}"
    response = call_api(url)
    if response is None:
        print(f"Failed to retrieve data for category {cat_id}")
        continue
    cat_data = response.json()['data']['categories'][0]['category']
    for challenge in cat_data['challenges']:
        challenge_row = {
            'category_id': cat_id,
            'category_title': cat_data['title'],
            'challenge_id': challenge['challenge']['id'],
            'challenge_title': challenge['challenge']['title'],
            'challenge_points': challenge['challenge']['points']
        }
        challenge_list.append(challenge_row)


In [ ]:
challenges_df = pd.DataFrame(challenge_list)
challenges_df.to_csv('challenges.csv', index=False)
print(f"Challenges saved to challenges.csv ({len(challenge_list)} challenges total)")
print(challenges_df.head())

In [ ]:
# učitati zadatke iz challenge.csv
challenges_df = pd.read_csv('challenges.csv')

U popis korisnika dodajem samo korisnike koji su riješili barem jedan zadatak

In [ ]:
# stanje users i solves tablice 11.9.2025.
# Prikupi info o svakom zadatku 419 poziva i stvaranje popisa korisnika
solve_nums = {}
users_set = set()
solves = []

#Dohvatiti sve ideve zadataka
challenge_ids = challenges_df['challenge_id'].tolist()

# Pomocu id-eva zadataka dohvatiti info o zadacima, korisnicim koji su ih rijesili i rjesenjima
for id in challenge_ids:
    url = f"https://ringzer0ctf.com/api/challenge/users/{id}"
    response = call_api(url)
    if response is None:
        print(f"Failed to retrieve data for challenge {id}")
        continue
    solve = response.json()['data']

    # za stvaranje solves_df = povezivanje korisnika i rjesenja zadataka
    solvers = solve['categories'][0]['category']['challenges'][0]['challenge']['Solvers']
    # u api dokumentaciji pise validation_time, ali je zapravo validationTime
    solves.extend([{
        "challenge_id": id,
        "user_id": s['Solver']['userId'],
        "validation_time": s['Solver']['validationTime']
    } for s in solvers])


    # za stvaranje users_df
    users_set.update([(u['user']['id'], u['user']['username'], u['user']['isRCEH'], u['user']['country']) for u in solve['users']])

    # za nadopunu broja rjesenja pojedinog zadatka u challenges_df
    solve_nums[id] = len(solve['users'])

In [ ]:
solve_df = pd.DataFrame(solves)
solve_df.head()
solve_df.to_csv('solves.csv', index=False)

In [ ]:
users_df = pd.DataFrame(list(users_set), columns=["id", "username", "isRCEH", "country"])
users_df.head()
users_df.to_csv('users.csv', index=False)

## Nadopuna Challenges tablice

In [ ]:
# challenges_df = pd.read_csv('challenges.csv')

In [ ]:
challenges_df["challenge_solves"] = challenges_df['challenge_id'].map(solve_nums)
challenges_df.head()

In [ ]:
subcategory_to_overarching = {}
for over_cat, subcats in categories.items():
    for subcat in subcats:
        subcategory_to_overarching[subcat] = over_cat
challenges_df["overarching_category"] = challenges_df["category_title"].map(subcategory_to_overarching)

In [ ]:
challenges_df.head()
challenges_df.to_csv('challenges1.csv', index=False)

## Nadopuna Users tablice

Sljedeći cell vrtila više puta za različite chunkove popisa da bih prikupila sve korisnike. 
prvi run do 4213
drugi run do 

Dodati stvari za bolje scrapeanje

In [33]:
users_df = pd.read_csv('users1.csv')

Sljedeći odlomak vrtila u više odlomaka jer na početku nisam pametno queryjala api

In [26]:
import json
import concurrent
from tqdm import tqdm

def fetch_user(id):
    url = f"https://ringzer0ctf.com/api/user/info/{id}"
    response = call_api(url)
    # dodati error handling
    try:
        user_info = response['data']['users'][0]['user']
        return({
            "id": id,
            "joined_date": user_info['memberSince'],
            "last_date": user_info['lastFlag'],
            "points": user_info['points'],
            "rank": user_info['rank']
        })
    except (json.JSONDecodeError, Exception) as e:
        print(f"Error retrieving user info for ID {id}: {e}")

user_data = []
ids = users_df['id'].iloc[15740:]

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(fetch_user, ids)
    for result in tqdm(results, total=len(ids)):
        if result is not None:
            user_data.append(result)


100%|██████████| 8773/8773 [10:22<00:00, 14.09it/s]


In [27]:
len(user_data)

8773

In [28]:
user_data

[{'id': 46062,
  'joined_date': '2023-07-31 07:56:44',
  'last_date': '2023-07-31 12:01:01',
  'points': '61',
  'rank': '1503'},
 {'id': 26265,
  'joined_date': '2019-01-13 07:11:33',
  'last_date': '2019-01-13 16:07:25',
  'points': '4',
  'rank': '13815'},
 {'id': 39788,
  'joined_date': '2021-07-25 02:52:08',
  'last_date': '2021-08-11 13:47:35',
  'points': '95',
  'rank': '876'},
 {'id': 51054,
  'joined_date': '2025-01-14 08:42:09',
  'last_date': '2025-01-14 08:50:21',
  'points': '2',
  'rank': '19763'},
 {'id': 2642,
  'joined_date': '2015-04-12 11:46:11',
  'last_date': '2015-04-12 12:30:12',
  'points': '1',
  'rank': '20193'},
 {'id': 48151,
  'joined_date': '2024-01-22 13:30:42',
  'last_date': '2024-01-22 13:54:21',
  'points': '3',
  'rank': '16769'},
 {'id': 8004,
  'joined_date': '2015-12-27 07:19:55',
  'last_date': '2016-01-08 05:42:45',
  'points': '27',
  'rank': '3374'},
 {'id': 8021,
  'joined_date': '2015-12-28 00:05:00',
  'last_date': '2016-02-08 00:08:50',
 

In [ ]:
user_update_df = pd.DataFrame(user_data)

users_df_indexed = users_df.set_index("id")
user_update_df_indexed = user_update_df.set_index("id")

# Combine, preferring non-null values from user_update_df
users_df_combined = users_df_indexed.combine_first(user_update_df_indexed).reset_index()
users_df_combined.head()

,id,country,isRCEH,joined_date,last_date,points,rank,username
0,6,Canada,False,2014-02-09 13:33:05,2016-02-10 23:23:06,118,661,ramsexy
1,10,Canada,False,2014-02-09 13:33:05,2014-02-06 20:20:24,3.0,14625.0,m6a2x6
2,11,Canada,False,2014-02-09 13:33:05,2014-02-23 02:17:15,18.0,4876.0,chaput
3,12,Canada,False,2014-02-09 13:33:05,2019-05-02 13:12:46,20.0,4472.0,mdube
4,14,Canada,False,2014-02-09 13:33:05,2014-09-04 17:44:52,55,1652,L30ballz


In [ ]:
users_df_combined.to_csv('users2.csv', index=False)
users_df = users_df_combined

# Analiza podataka

In [ ]:
challenges_df = pd.read_csv('challenges1.csv')
users_df = pd.read_csv('users2.csv')
solve_df = pd.read_csv('solves.csv')

Prosječna riješenost po kategoriji za sve korisnike

In [ ]:
challenges_df.head()

In [ ]:
avg_solve_rate_per_subcategory = challenges_df.groupby('category_title')['challenge_solves'].mean().reset_index()

In [ ]:
print(avg_solve_rate_per_subcategory.sort_values(by='challenge_solves', ascending=False))

A koje su to kategorije kad razmišljamo o standardnim jeopardy kategorijama?

In [ ]:
avg_solve_rate_per_category = challenges_df.groupby('overarching_category')['challenge_solves'].mean().reset_index()
print(avg_solve_rate_per_category.sort_values(by='challenge_solves', ascending=False))

Koji su najuspješniji korisnici i koliko im je dugo trebalo da dođu do trenutnog ranga?

Za izračunati period naći zadnji flag koji su submitali? Updated users table iznad tim informacijama